# 01 Transport Data
This notebook contains code to parse data from experiment set-up to analytics section.

In [1]:
import shutil
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML

In [5]:
experiment_name = "experiment_06"

In [6]:
def transport_logs(experiment_name, only_docker=True):
    source_native = Path("../../kex-native/logs")
    source_docker = Path("../../kex-docker/logs")

    target_native = Path("../data/native") / experiment_name
    target_docker = Path("../data/docker") / experiment_name

    if not only_docker:
        target_native.mkdir(parents=True, exist_ok=True)
    target_docker.mkdir(parents=True, exist_ok=True)

    if not only_docker:
        if source_native.exists():
            shutil.copytree(source_native, target_native, dirs_exist_ok=True)
            print(f"Native logs copied to {target_native}")
        else:
            print(f"Source not found: {source_native}")

    if source_docker.exists():
        shutil.copytree(source_docker, target_docker, dirs_exist_ok=True)
        print(f"Docker logs copied to {target_docker}")
    else:
        print(f"Source not found: {source_docker}")



In [7]:
transport_logs(experiment_name)

Docker logs copied to ../data/docker/experiment_06


Data preview

In [8]:
def preview_data(experiment_name, side = "client", only_docker=True):
    if not only_docker:
        base_native = Path("../data/native") / experiment_name
    base_docker = Path("../data/docker") / experiment_name
    modes = ["classic", "pqc", "hybrid"]
    metrics = ["duration_sec", "cpu_percent", "ram_percent"]

    if side == "client":
        csv_name = "client_log.csv"
    elif side == "server":
        csv_name = "server_log.csv"
    else:
        raise ValueError("side must be 'client' or 'server'")

    def aggregate(path, agg_func, csv_name):
        rows = []
        for mode in modes:
            file_path = path / mode / csv_name
            if file_path.exists():
                df = pd.read_csv(file_path)
                if not df.empty:
                    values = df[metrics].agg(agg_func)
                    rows.append(values)
                else:
                    rows.append(pd.Series({k: None for k in metrics}))
            else:
                rows.append(pd.Series({k: None for k in metrics}))
        result_df = pd.DataFrame(rows, index=modes)
        return result_df

    if not only_docker:
        native_mean = aggregate(base_native, "mean", csv_name)
        native_median = aggregate(base_native, "median", csv_name)
    docker_mean = aggregate(base_docker, "mean", csv_name)
    docker_median = aggregate(base_docker, "median", csv_name)

    html_parts = ["<div style='display: flex; flex-wrap: wrap; gap: 40px;'>"]

    if not only_docker:
        html_parts.append(f"""
            <div>
                <h4>Native – Mean</h4>
                {native_mean.to_html()}
            </div>
            <div>
                <h4>Native – Median</h4>
                {native_median.to_html()}
            </div>
        """)

    html_parts.append(f"""
        <div>
            <h4>Docker – Mean</h4>
            {docker_mean.to_html()}
        </div>
        <div>
            <h4>Docker – Median</h4>
            {docker_median.to_html()}
        </div>
    </div>
    """)

    display(HTML("".join(html_parts)))


In [9]:
preview_data(experiment_name, side="client")

,duration_sec,cpu_percent,ram_percent
classic,0.344465,0.2700,8.7179
pqc,0.365732,0.2752,8.7665
hybrid,0.353026,0.2881,9.2408
,duration_sec,cpu_percent,ram_percent
classic,0.308173,0.3,8.7
pqc,0.308534,0.3,8.8
hybrid,0.308863,0.3,9.2


In [10]:
preview_data(experiment_name, side="server")

,duration_sec,cpu_percent,ram_percent
classic,0.167796,0.2729,8.7179
pqc,0.169247,0.2783,8.7665
hybrid,0.170687,0.2919,9.2408
,duration_sec,cpu_percent,ram_percent
classic,0.154506,0.3,8.7
pqc,0.154600,0.3,8.8
hybrid,0.155509,0.3,9.2
